In [ ]:
import os
import pandas
import json
import folium
import numpy as np

In [ ]:
cantons_geo_path = r'topojson/ch-cantons.topojson.json'
cantons_json_data = json.load(open(cantons_geo_path))

In [ ]:
europe_geo_path = r'topojson/europe.topojson.json'
europe_json_data = json.load(open(europe_geo_path))

In [ ]:
for key in cantons_json_data['objects']:
    print("key: %s , " % key)


# cantons_json_data keys: 'type', 'transform', 'objects', 'arcs'
# cantons_json_data['type'] = Topology
# cantons_json_data['transform'] keys: 'scale', 'translate'
# cantons_json_data['objects'] keys: cantons
# cantons_json_data['arcs'] keys: boundaries for each canton




> **Short Description** 

Unemployment rates represent unemployed persons as a percentage of the labour force. The labour force is the total number of people employed and unemployed. Unemployed persons comprise persons aged 15 to 74 who were: a. without work during the reference week, b. currently available for work, i.e. were available for paid employment or self-employment before the end of the two weeks following the reference week, c. actively seeking work, i.e. had taken specific steps in the four weeks period ending with the reference week to seek paid employment or self-employment or who found a job to start later, i.e. within a period of, at most, three months. This table does not only show unemployment rates but also unemployed in 1000 and as % of the total population.

In [ ]:
io = 'Eurostat_Table_Active_Population.xls'
na_values= [':', '(i)', '(p)']
df = pandas.read_excel(io, skiprows=range(3), skip_footer=9, 
                          na_values=na_values, header=0)


df = df.rename(columns={'geo\\time': 'country'})
df = df.set_index('country')
df = df.iloc[4:, 0:len(df.columns):2]

df.head()

In [ ]:
q = list(df['2016'].quantile([.05, .235, .655, .774, .9455]))

geometries = []

c_json = []
for key in europe_json_data['objects']['europe']['geometries']:
    country = key['properties']['NAME']
    c_json.append(country)
    if country in df.index:
        geometries.append(key)
        
europe_json_data['objects']['europe']['geometries'] = geometries

In [ ]:
m_europe = folium.Map(location=[46, 15], tiles='cartodbpositron', zoom_start =4)
m_europe.choropleth(geo_data=europe_json_data, data=df.reset_index(),
                columns = ['country', '2016'],
                threshold_scale = q,
                key_on='feature.properties.NAME',
                topojson='objects.europe',
                fill_color='YlGn',
                fill_opacity=0.7,
                line_opacity=0.2,
                legend_name='Unemployment Rate (%)')

m_europe

In [ ]:
io = 'Eurostat_Table_Total_Population.xls'
na_values= [':', '(i)', '(p)']
df = pandas.read_excel(io, skiprows=range(3), skip_footer=9, 
                          na_values=na_values, header=0)


df = df.rename(columns={'geo\\time': 'country'})
df = df.set_index('country')
df = df.iloc[4:, 0:len(df.columns):2]

df.loc[:, '1999':'2016'].head()

In [ ]:
io = 'Unemployment_Rates_CH_2016.xlsx'
df = pandas.read_excel(io, skiprows=range(2), skip_footer=1, header=0)


df = df.loc[:, ['Canton', 'Total']]
df = df.drop(df.index[0])
df = df.reset_index().drop('index', axis=1)
df['Total'] = df['Total'].astype('float64', copy=False, errors='ignore')

ids = []
for canton in cantons_json_data['objects']['cantons']['geometries']:
    ids.append(canton['id'])

df['ID'] = pandas.Series(ids, name='IDs', dtype=str, copy=False)

df.head()

In [ ]:
q = list(df['Total'].quantile([0, .15, .4, .72, .875, 1]))
q

In [ ]:
m_switzerland = folium.Map(location=[47.1, 8.39], tiles='cartodbpositron', zoom_start =7)
m_switzerland.choropleth(geo_data=cantons_json_data, data=df,
                columns = ['ID', 'Total'],
                threshold_scale = q,
                key_on='feature.id',
                topojson='objects.cantons',
                fill_color='YlGn',
                fill_opacity=0.7,
                line_opacity=0.2,
                legend_name='Unemployment Rate Switzerland (%)')

m_switzerland